In [1]:
import pickle, json, time, datetime, pymongo, requests, copy, os, csv, math
from pymongo import MongoClient
import pandas as pd
from Bio import Medline
from Bio import Entrez

In [2]:
now=datetime.datetime.now()

In [8]:
nowtime=now.strftime('%Y/%m/01')

In [9]:
print(nowtime)

2023/12/01


In [6]:
now_time=now.strftime('%Y_%m_01')

In [7]:
print(now_time)

2023_12_01


In [10]:
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 12
    day = int(time[2])
    if month < 1:
        month += 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [11]:
def cal_time2(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) +1
    day = int(time[2])
    if month >12:
        month = 1
        year += 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [12]:
former_time=cal_time(nowtime)

In [13]:
former_time

'2022/12/01'

In [14]:
idlist=[]
while former_time < now_time:
    terms='("aging"[Title/Abstract] OR "ageing"[Title/Abstract] OR "longevity"[Title/Abstract] OR "centenarian"[Title/Abstract] OR "centenarians"[Title/Abstract] \
            OR "the elderly"[Title/Abstract] OR "the aged"[Title/Abstract] OR "old people"[Title/Abstract] OR "older people"[Title/Abstract] OR "old age"[Title/Abstract] \
            OR "gerontology"[Title/Abstract] OR "geroscience"[Title/Abstract] OR "lifespan"[Title/Abstract] OR "healthspan"[Title/Abstract] OR "life expectancy"[Title/Abstract] AND \
            Journal Article[ptyp] AND "humans"[MeSH Terms] AND English[lang] AND ("'+str(former_time)+'"[Date - Publication] : "'+str(cal_time2(former_time))+'"[Date - Publication]))'
    Entrez.email="12007002@zju.edu.cn"
    handle_0 = Entrez.esearch(db="pubmed", term=terms,retmax=10000)
    record = Entrez.read(handle_0)
    idlist+=list(record["IdList"])
    # print("Found %i results" % len(idlist))
    if len(list(record["IdList"]))==9999:
        print('error')
        break
    former_time=cal_time2(former_time)

In [15]:
len(idlist)

40154

In [16]:
len(set(idlist))

20644

In [17]:
idlist_list=list(set(idlist))

In [18]:
with open('idlist.txt','a') as f:
    for i in idlist_list:
        f.write(i)

In [19]:
client = MongoClient(username="hald",password="mclab236")

In [20]:
db = client.hald

In [21]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hald')

# pubmed_raw_pmid

In [22]:
collections_pubmed_raw_pmid=db.pubmed_raw_pmid

In [23]:
pmids=list(collections_pubmed_raw_pmid.find())

In [24]:
len(pmids)

339145

In [25]:
pmids[:2]

[{'_id': ObjectId('6393476d5303aaecac1f079a'),
  'PMID': '35796512',
  'addtime': '20220724'},
 {'_id': ObjectId('6393476d5303aaecac1f079b'),
  'PMID': '35796396',
  'addtime': '20220724'}]

In [26]:
pmid_list=[]
for i in pmids:
    pmid_list.append(i.get('PMID'))

In [27]:
pmid_list[:2]

['35796512', '35796396']

In [28]:
len(pmid_list)

339145

In [29]:
len(pmid_list)

339145

In [30]:
pmid_dict={}
for i in pmids:
    pmid_dict[i.get('PMID')]=i.get('PMID')

In [31]:
len(pmid_dict)

339145

In [32]:
new_articles_raw=[]
for i in idlist_list:
    if not pmid_dict.get(i):
        new_articles_raw.append(i)

In [33]:
len(new_articles_raw)

3743

In [34]:
res=[]
for i in range(0,len(new_articles_raw),1000):
    id=new_articles_raw[i:i+1000]
    handle_1 = Entrez.efetch(db="pubmed", id=id, rettype="medline",retmode="text")
    records = Medline.parse(handle_1)
    res.append(list(records))

In [35]:
raw_pick=[]
for i in res:
    raw_pick+=i

In [36]:
len(raw_pick)

3743

In [37]:
reserved_raw_pick=[]
reserved_raw_pick_pmid=[]
for i in raw_pick:
    if i.get('AB'):
        if not pmid_dict.get(i.get('PMID')):
            i['addtime']=now_time
            reserved_raw_pick.append(i)
            reserved_raw_pick_pmid.append({'PMID':i.get('PMID'),'addtime':i.get('addtime')})

In [38]:
len(reserved_raw_pick)

3506

In [39]:
reserved_raw_pick[0]

{'PMID': '37881145',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20231115',
 'LR': '20231115',
 'IS': '1559-2863 (Electronic) 0361-7734 (Linking)',
 'VI': '48',
 'IP': '6',
 'DP': '2023 Nov 1',
 'TI': 'Effect of Tooth Bleach on Dentin Fatigue Resistance in Situ.',
 'PG': '638-647',
 'LID': '10.2341/22-093-C [doi]',
 'AB': 'BACKGROUND: Negative effects of bleaching on dentin have previously been reported in vitro. OBJECTIVE: The purpose of this study was to determine the effect of carbamide peroxide bleaching on dentin fatigue resistance using a clinically relevant in situ model. METHODS AND MATERIALS: Following research ethics board approval, 60 human teeth requiring extraction were collected. Sterilized human dentin specimens were cut (1.2x1.2x10 mm) and secured into customized bleaching trays to be used by study participants. Participants were randomly assigned to either bleach (10% carbamide peroxide, n=23) or control (gel without bleach, n=26) treatment groups. Treatment was appli

In [40]:
collections_pubmed_raw_pmid.insert_many(reserved_raw_pick_pmid)

In [41]:
len(list(collections_pubmed_raw_pmid.find()))

342651

In [45]:
reserved_raw_pick_pmid[-1]

{'PMID': '37875871',
 'addtime': '2023_12_01',
 '_id': ObjectId('65757b1d22f3b73537601bdd')}

# pubmed_raw (reserved_raw_pick)

In [46]:
collections_pubmed_raw = db.pubmed_raw

In [47]:
collections_pubmed_raw.insert_many(reserved_raw_pick)

In [48]:
reserved_raw_pick[20]

{'PMID': '38042773',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20231204',
 'LR': '20231205',
 'IS': '1472-6963 (Electronic) 1472-6963 (Linking)',
 'VI': '23',
 'IP': '1',
 'DP': '2023 Dec 2',
 'TI': "Effect of robot for medication management on home care professionals' use of working time in older people's home care: a non-randomized controlled clinical trial.",
 'PG': '1344',
 'LID': '10.1186/s12913-023-10367-0 [doi] 1344',
 'AB': "BACKGROUND: Medication management has a key role in the daily tasks of home care professionals delivered to older clients in home care. The aim of this study was to examine the effect of using a robot for medication management on home care professionals  use of working time. METHODS: A pragmatic non-randomized controlled clinical trial was conducted. The participants were home care professionals who carried out home care clients' medication management. Home care clients were allocated into intervention groups (IG) and control groups (CG) (n = 64 and 46, 

# pubtator_raw_pmid

In [49]:
collection_pubtator_raw_pmid = db.pubtator_raw_pmid

In [50]:
previous_pubtator_pmid=list(collection_pubtator_raw_pmid.find({}))

In [51]:
len(previous_pubtator_pmid)

316859

In [52]:
previous_pubtator_pmids_dict={}
for i in previous_pubtator_pmid:
    previous_pubtator_pmids_dict[i.get('PMID')]=i.get('PMID')

In [53]:
current_pubtator_pmid=[]
for i in reserved_raw_pick:
    if i.get('AB'):
        if not previous_pubtator_pmids_dict.get(i.get('PMID')):
            i['addtime']=now_time
            current_pubtator_pmid.append(i.get('PMID'))

In [54]:
len(current_pubtator_pmid)

3506

In [55]:
pmids_list=[]
for i in current_pubtator_pmid:
    pmids_list.append(i)

In [56]:
len(pmids_list)

3506

In [57]:
def SubmitPMIDList(pmids,Format,now_time):
	json = {"pmids": pmids}
	r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/"+Format , json = json)
	if r.status_code != 200 :
		print ("[Error]: HTTP code "+ str(r.status_code))
	else:
		with open(now_time+'_pubtator_raw.txt','a',encoding="UTF-8") as fileread:
			fileread.write(str(r.text.encode("utf-8"),encoding="UTF-8"))

In [58]:
import time
pubtator_raw=[]
for i in range(0,len(pmids_list),1000):
    SubmitPMIDList(pmids_list[i:i+1000],'biocjson',now_time)
    time.sleep(0.1)

In [59]:
ttt=open(now_time+'_pubtator_raw.txt',"r")

In [60]:
overjson=[]
for l in ttt.readlines():
    overjson.append(l.rstrip())

In [61]:
len(overjson)

3423

In [62]:
pubtator_list=copy.deepcopy(overjson)

In [63]:
pubtator_json=[]
for i in pubtator_list:
    pubtator_json.append(json.loads(i))

In [64]:
pubtator_pmid_list=[]
for i in pubtator_json:
    pubtator_pmid_list.append(i.get('id'))

In [65]:
len(pubtator_pmid_list)

3423

In [66]:
pubtator_pmid_list[0]

'38048153'

In [67]:
current_pubtator_pmid_list=[]
for i in pubtator_pmid_list:
    current_pubtator_pmid_list.append({'PMID':i,'addtime':now_time})

In [68]:
len(current_pubtator_pmid_list)

3423

In [69]:
collection_pubtator_raw_pmid.insert_many(current_pubtator_pmid_list)

# pubtator_info  (raw_pick_pubtator)

In [70]:
collections_pubtator_info = db.pubtator_info

In [71]:
filtered_pubtator_json=[]
for i in pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                if j.get('infons').get('identifier')=='9606':
                    filtered_pubtator_json.append(i)
                    break

In [72]:
len(filtered_pubtator_json)

2646

In [73]:
second_filtered_pubtator_json=[]
for i in filtered_pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                    if j.get('infons').get('type') in ('Gene', 'Chemical', 'Mutation','Disease') :
                        second_filtered_pubtator_json.append(i)
                        break

In [74]:
len(second_filtered_pubtator_json)

2302

In [75]:
entities_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        entities_list=[]
        for j in i.get('passages')[1].get('annotations'):
            entities_list.append(j.get('text'))
        entities_dict[i.get('id')]=entities_list

In [76]:
second_filtered_pubtator_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        second_filtered_pubtator_dict[i.get('id')]=i

In [77]:
second_filtered_pubtator_json[0]

{'_id': '38048153|None',
 'id': '38048153',
 'infons': {},
 'passages': [{'infons': {'journal': 'J Med Internet Res;2023Dec04; 25 49237. doi:10.2196/49237',
    'year': '2023',
    'type': 'title',
    'authors': 'Zhou S, Gromala D, Wang L, ',
    'section': 'Title'},
   'offset': 0,
   'text': 'Ethical Challenges of Virtual Reality Technology Interventions for the Vulnerabilities of Patients With Chronic Pain: Exploration of Technician Responsibility.',
   'sentences': [],
   'annotations': [{'id': '2',
     'infons': {'identifier': '9606', 'type': 'Species'},
     'text': 'Patients',
     'locations': [{'offset': 90, 'length': 8}]},
    {'id': '3',
     'infons': {'identifier': 'MESH:D059350', 'type': 'Disease'},
     'text': 'Chronic Pain',
     'locations': [{'offset': 104, 'length': 12}]}],
   'relations': []},
  {'infons': {'type': 'abstract', 'section': 'Abstract'},
   'offset': 160,
   'text': 'Chronic pain, a common disease, is a crucial global public health concern. Approxima

In [78]:
from nltk.tokenize import sent_tokenize
import datetime
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,rawlength,lock):
    info=i.get('passages')[1]
    abstract=info.get('text')
    sentences=sent_tokenize(abstract)
    positions=[]
    size=len(sentences)
    start=0
    for k,j in enumerate(sentences):
        newj=j.translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        spanlist=re.search(newj,abstract,flags=re.M|re.I).span()
        abstract=abstract[spanlist[1]:]
        positions.append((start+spanlist[0],start+spanlist[1]))
        start+=spanlist[1]
    middict={'PMID':i.get('id'),'AB':info.get('text'),'SENTENCE':sentences,'positions':positions,'annotations':info.get('annotations'),'offset':info.get('offset')}
    lock.acquire()
    rawlength.append(middict)
    lock.release()
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    rawlength=manager.list()
    lock=manager.Lock()
    for i in second_filtered_pubtator_json:
        if i.get('passages'):
            p.apply_async(work, args=(i,rawlength,lock))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 99387.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 13s


In [79]:
pubtators_list=list(rawlength)

In [80]:
len(pubtators_list)

2302

In [81]:
pubtator_raw=[]
error=[]
error2=[]
for i in pubtators_list:
    for j in i.get('annotations'):
        for k,l in enumerate(i.get('positions')):
            if j.get('locations')[0].get('offset')>=l[0]+i.get('offset') and j.get('locations')[0].get('offset')<l[1]+i.get('offset'):
                pubtator_raw.append({'id':i.get('PMID'),'AB':i.get('AB'), 'type': j.get('infons').get('type'), \
                                     'identifier':j.get('infons').get('identifier'),'text':j.get('text'),'sentence':i.get('SENTENCE')[k],\
                                     'sentenceSize':len(i.get('SENTENCE')),'sentenceIndex':k})
                break
        else:
            error.append(i)
            error2.append(j)

In [82]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

In [83]:
len(error)

0

In [84]:
len(error2)

0

In [85]:
len(pubtator_raw)

33030

In [86]:
raw_pick_pubtator=[]
for i in reserved_raw_pick:
    if i.get('AB'):
        if pubtator_raw_dict2AB.get(i.get('PMID')):
            i.update({'AB':pubtator_raw_dict2AB.get(i.get('PMID'))})
            raw_pick_pubtator.append(i)

In [87]:
len(pubtator_raw_dict2AB)

2302

In [88]:
len(reserved_raw_pick)

3506

In [89]:
len(raw_pick_pubtator)

2302

In [90]:
for i in raw_pick_pubtator:
    i['addtime']=now_time

In [91]:
collections_pubtator_info.insert_many(raw_pick_pubtator)

In [92]:
len(list(collections_pubtator_info.find()))

208697

In [93]:
len(list(collections_pubtator_info.find()))

208697

# pubtator_for_compare_latest (pubtator_raw)

In [94]:
collections_pubtator_for_compare_latest=db.pubtator_for_compare_latest

In [95]:
for i in pubtator_raw:
    i['addtime']=now_time

In [96]:
collections_pubtator_for_compare_latest.insert_many(pubtator_raw)

In [97]:
len(pubtator_raw)

33030

In [98]:
pwd

'/root/hald_autoupdate/2023_12_10'

In [99]:
os.makedirs('results/step1/')
os.makedirs('results/step2/')
os.makedirs('results/step3/')
os.makedirs('results/step4/')
os.makedirs('results/step5/')

In [100]:
pickle.dump(raw_pick,open('results/step1/raw_pick.pkl','wb'))
pickle.dump(raw_pick_pubtator,open('results/step1/raw_pick_pubtator.pkl','wb'))
pickle.dump(pubtator_raw,open('results/step1/pubtator_raw.pkl','wb'))

In [101]:
pubtator_raw[0]

{'id': '38036596',
 'AB': 'Epidemiological studies have reported a strong association between circulating Klotho and physical function; however, the cohorts were comprised of older adults with multiple comorbidities. Herein, we examined the relationship between Klotho and physical function in a community-based cohort of healthy adults. In this cross-sectional study, serum Klotho was measured in 80 adults who visited the Musculoskeletal Function, Imaging, and Tissue Resource Core of the Indiana Center for Musculoskeletal Health. Participants (n = 20, 10 [50%] men per group) were chosen into four age groups: 20-34, 35-49, 50-64, and >= 65 years, and were further grouped based on performance (low vs. high) on grip strength and chair stand tests. Klotho levels were lower in the >= 65 years group (703.0 [189.3] pg/mL; p = 0.022) and the 50-64 years group (722.6 [190.5] pg/mL; p = 0.045) compared to 20-34 years (916.1 [284.8] pg/mL). No differences were observed in Klotho between the low and

In [102]:
now_time

'2023_12_01'

# Literature_Info.json

In [103]:
pd.set_option('display.max_columns', 50)

In [105]:
jcr=pd.read_csv("../JCR/jcr2023.tsv",sep="\t")

In [106]:
len(jcr)

20522

In [107]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
20512,9855,European Journal of Orthopaedic Surgery and Tr...,EUR J ORTHOP SURG TR,2023,NaN,1633-8065,3574,1.7,1.6,NaN,0.3,326,4.7,8.3,0.00518,NaN,0,0,0,0,0,NaN,1.11597
20513,9855,European Journal of Transport and Infrastructu...,EUR J TRANSP INFRAST,2023,NaN,1567-7133,664,1.7,1.7,1.5,0.5,19,8.4,6.2,0.00036,0.336,0,0,0,0,0,9.5,0.07776
20514,9855,European Journal on Criminal Policy and Research,EUR J CRIM POLICY RE,2023,NaN,0928-1371,773,1.7,1.6,1.9,0.5,35,6.8,8.0,0.00092,0.630,0,0,0,0,0,44.2,0.19839
20515,9855,EXPERIMENTAL LUNG RESEARCH,EXP LUNG RES,2023,NaN,0190-2148,1340,1.7,1.7,2.0,0.3,24,9.5,6.0,0.00067,0.379,0,0,0,0,0,8.5,0.14451
20516,9855,FEDERAL RESERVE BANK OF ST LOUIS REVIEW,FED RESERVE BANK ST,2023,NaN,0014-9187,576,1.7,1.6,1.4,0.4,18,13.5,6.7,0.00069,0.857,0,0,0,0,0,40.1,0.14960
20517,9855,Feminist Media Studies,FEM MEDIA STUD,2023,NaN,1468-0777,2273,1.7,1.4,2.3,0.3,175,4.9,8.2,0.00502,1.025,0,0,0,0,0,55.7,1.08187
20518,9855,FIBER AND INTEGRATED OPTICS,FIBER INTEGRATED OPT,2023,NaN,0146-8030,265,1.7,1.7,1.0,0.3,8,8.8,6.4,0.00013,0.161,0,0,0,0,0,27.5,0.02988
20519,9855,Field Methods,FIELD METHOD,2023,NaN,1525-822X,3624,1.7,1.6,3.9,0.3,41,16.3,9.4,0.00150,1.336,0,0,0,0,0,67.6,0.32450
20520,9855,FINANCE AND STOCHASTICS,FINANC STOCH,2023,NaN,0949-2984,1384,1.7,1.6,2.3,0.3,31,12.7,13.2,0.00202,1.563,0,0,0,0,0,59.6,0.43612


In [17]:
jt2if_all={}
for i in range(len(jcr)):
    if jcr.iloc[i,7]=='<0.1':
        jcr.iloc[i,7]='0.0'
    if math.isnan(float(jcr.iloc[i,7])):
        jt2if_all[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=float(jcr.iloc[i,7])
jt2if_all['aging']=jt2if_all.get('aging-us')

In [18]:
jt2if_brief={}
for i in range(len(jcr)):
    if jcr.iloc[i,7]=='<0.1':
        jcr.iloc[i,7]='0.0'
    if math.isnan(float(jcr.iloc[i,7])):
        jt2if_brief[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=float(jcr.iloc[i,7])
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [19]:
jt2if5_all={}
for i in range(len(jcr)):
    if jcr.iloc[i,9]=='<0.1':
        jcr.iloc[i,9]='0.0'
    if math.isnan(float(jcr.iloc[i,9])):
        jt2if5_all[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=float(jcr.iloc[i,9])
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [20]:
jt2if5_brief={}
for i in range(len(jcr)):
    if jcr.iloc[i,9]=='<0.1':
        jcr.iloc[i,9]='0.0'
    if math.isnan(float(jcr.iloc[i,9])):
        jt2if5_brief[jcr.iloc[i,1].lower()]=0.0
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=float(jcr.iloc[i,9])
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [114]:
Literature_Info_list=list(collections_pubmed_raw.find())

In [115]:
len(Literature_Info_list)

342651

In [116]:
Literature_Info_dict={}
for i in Literature_Info_list:
    if jt2if_brief.get(i.get('TA').lower()):
        i['IF']=jt2if_brief.get(i.get('TA').lower())
    elif jt2if_all.get(i.get('JT').lower()):
        i['IF']=jt2if_all.get(i.get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(i.get('TA').lower()):
        i['IF5']=jt2if5_brief.get(i.get('TA').lower())
    elif jt2if5_all.get(i.get('JT').lower()):
        i['IF5']=jt2if5_all.get(i.get('JT').lower())
    else:
        i['IF5']=0.0
    Literature_Info_dict[i.get('PMID')]={
    'PMID':i.get('PMID'),
        'TI':i.get('TI'),
        'AB':i.get('AB'),
        'IF':i.get('IF'),
        'IF5':i.get('IF5'),
        'AU':i.get('AU'),
        'FAU':i.get('FAU'),
        'AD':i.get('AD'),
        'PT':i.get('PT'),
        'DP':i.get('DP'),
        'PL':i.get('PL'),
        'JT':i.get('JT'),
        'TA':i.get('TA'),
        'SO':i.get('SO')
    }

In [117]:
list(Literature_Info_dict.items())[10]

('35795148',
 {'PMID': '35795148',
  'TI': 'Sirtuin 1 in Chronic Kidney Disease and Therapeutic Potential of Targeting Sirtuin 1.',
  'AB': 'The incidence and prevalence of chronic kidney disease (CKD) continue to increase worldwide remaining as a major public health burden. CKD eventually progresses to end-stage kidney failure and patients with CKD have high morbidity and mortality. Sirtuin 1 (SIRT1), a NAD+-dependent deacetylases, has significant renal protective effects through its regulation of fibrosis, apoptosis, and senescence, oxidative stress, inflammation and aging process. The renal protective effects of Sirt1 have been described in many kidney diseases such as diabetic kidney disease and HIV-related kidney disease. SIRT1 also has protective effects against vascular calcification and therefore could be developed as a therapy for both CKD and CKD complications. In this narrative review, we will give an overview of the recent progress on the role of SIRT1 and its downstream pa

In [118]:
json.dump(Literature_Info_dict,open('results/step5/Literature_Info.json','w'))

In [119]:
len(Literature_Info_dict)

342651